In [ ]:
import threading
import numpy as np
import cv2
from pykinect2 import PyKinectRuntime
from pykinect2.PyKinectV2 import FrameSourceTypes_Color
from PointCloud import Cloud


In [ ]:
# import sys
# sys.path.append('..\\..\\CV Model\\')
# from finetune import create_model

In [ ]:
import torch
from torchvision.transforms import functional as F
from torchvision.models.detection import fasterrcnn_resnet50_fpn

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


In [ ]:
# COCO class labels for torchvision models
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

In [ ]:
def load_model():
    # model = create_model(3)  # Number of classes
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using {device}")
    
    # model_path = '..\\..\\CV Model\\chkpt\\model.pth'
    # model.load_state_dict(torch.load(model_path, map_location=device))
    # model = torch.load("..\\..\\CV Model\\chkpt\\model.pth")
    model = fasterrcnn_resnet50_fpn(pretrained=True)
    model.to(device) 
    model.eval()
    return model, device


def process_frame(frame, model, device):
    # Resize frame to make processing more efficient
    # resized_frame = cv2.resize(frame, (800, 800))

    # Transform the frame to tensor and scale it
    frame_tensor = F.to_tensor(frame)
    frame_tensor = frame_tensor.to(torch.float32).to(device)

    # Correctly format the tensor as a list of 3D tensors [C, H, W]
    frame_tensor = frame_tensor.unsqueeze(0)  # Add a batch dimension if it's not already there

    # Ensure the tensor is passed as a list of images 
    frame_tensor_list = [frame_tensor.squeeze(0)]  # This changes it from [1, 3, H, W] to [3, H, W] and wraps in a list

    with torch.no_grad():
        predictions = model(frame_tensor_list)[0] 

    return predictions


def display_predictions(frame, predictions):
    # # Extract data from predictions
    # labels = predictions['labels']
    # boxes = predictions['boxes']
    # masks = predictions['masks']  # If using Mask R-CNN

    # # Draw boxes and labels on the frame here, or use utility functions from torchvision
    # frame_with_boxes = frame  # Placeholder for actual drawing function


    # Resize frame to the original display size
    # display_frame = cv2.resize(frame, (1920, 1080))
    
    #########
    # labels = predictions['labels']
    # boxes = predictions['boxes']
    # scores = predictions['scores']

    # for box, label, score in zip(boxes, labels, scores):
    #     if score > 0.85:  # Display only detections with confidence > 0.85
    #         cv2.rectangle(frame, 
    #                       (int(box[0]), int(box[1])), 
    #                       (int(box[2]), int(box[3])), 
    #                       (0, 255, 0), 2)  # Draw rectangle
    #         cv2.putText(frame, 
    #                     f'{label} {score:.2f}', 
    #                     (int(box[0]), int(box[1] - 10)), 
    #                     cv2.FONT_HERSHEY_SIMPLEX, 
    #                     0.5, (0, 255, 0), 2)  # Display label and score

    # return frame

    labels = predictions['labels']
    boxes = predictions['boxes']
    scores = predictions['scores']

    for box, label, score in zip(boxes, labels, scores):
        if score > 0.85:  # Display only detections with confidence > 0.5
            x1, y1, x2, y2 = map(int, box)
            label_name = COCO_INSTANCE_CATEGORY_NAMES[label.item()]
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw rectangle
            cv2.putText(frame, f'{label_name}: {score:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)  # Display label and score

    return frame

In [ ]:
def rgb_capture(kinect, model, device):
    print("Starting color frame capture. Press 'q' to quit.")

    while True:
        if kinect.has_new_color_frame():
            color_frame = kinect.get_last_color_frame()
            color_image = color_frame.reshape((kinect.color_frame_desc.Height,
                                               kinect.color_frame_desc.Width, 4)).astype(np.uint8)
            
            rgb_image = cv2.cvtColor(color_image, cv2.COLOR_RGBA2RGB)

            # Process frame through model
            predictions = process_frame(rgb_image, model, device)
            rgb_image_with_preds = display_predictions(rgb_image, predictions)

            cv2.imshow('Kinect RGB Stream with CNN Predictions', rgb_image_with_preds)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            # print("Waiting for new frame...")
            pass
        

In [ ]:
def point_cloud_visualization():
    pcl = Cloud(dynamic=True, simultaneously=True, depth=True, color=True, body=False, skeleton=False, color_overlay=False)
    # pcl.visualize()
    

In [ ]:
def main():
    # Initialize the Kinect with color and depth frame support
    kinect = PyKinectRuntime.PyKinectRuntime(FrameSourceTypes_Color)
    model, device = load_model()

    # Start the RGB capture thread
    rgb_thread = threading.Thread(target=rgb_capture, args=(kinect, model, device))
    rgb_thread.start()

    # Start the point cloud visualization in the main thread
    point_cloud_visualization()

    # Wait for the RGB thread to finish
    rgb_thread.join()

    # Cleanup
    kinect.close()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
    

In [ ]:
# import torch
# import torchvision
# print(torch.__version__)
# print(torchvision.__version__)
